In [88]:
!pip install dash
!pip install dash-html-components
!pip install dash-core-components
!pip install dash-table

In [89]:
!pip install pandas
!pip install numpy

In [90]:
import pandas as pd
import numpy as np

In [148]:
import dash
import pandas as pd
from dash import dcc, html, Input, Output, callback

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

def combine_names(row):
    return f"{row['Фамилия']} {row['Имя']} {row['Отчество']}"

g_persons_df = pd.read_csv('./db_persons.csv')[['ID', 'Фамилия', 'Имя', 'Отчество']]
g_persons_df['ФИО'] = g_persons_df.apply(combine_names, axis=1)
gpersons = g_persons_df[['ФИО', 'ID']]
gpersons.columns = ['label', 'value']
gpersons = gpersons.to_dict(orient='records')

app.layout = html.Div([
    html.Label('Выберите автора'),
    dcc.Dropdown(
        options=gpersons,
        placeholder="Выберите автора",
        id='author',
    ),
    html.Div(id='selected-author'),  # Для вывода выбранного значения
    dcc.Dropdown(id='works-dropdown', 
                 multi=True,
                 optionHeight=40)  # Для списка "Работы"
], style={'columnCount': 1})

# Глобальная переменная для хранения значения автора
selected_author_value = None

# Функция обратного вызова для обновления выбранного автора
@app.callback(
    Output('selected-author', 'children'),
    Input('author', 'value')
)
def update_selected_author(selected_value):
    global selected_author_value
    selected_author_value = selected_value
    
    if selected_value is None:
        return "Выберите автора"
    else:
        return f"Выбран автор: {selected_value}"

@app.callback(
    Output('works-dropdown', 'options', allow_duplicate=True),
    Output('works-dropdown', 'value', allow_duplicate=True),
    Input('works-dropdown', 'value'),
    prevent_initial_call=True
)
def update_selected_author(selected_value):
    global selected_author_value, a

    if selected_value is None:
        return [], []
    if selected_value.count('Все работы'):
        return [{'label': "Все работы", 'value': 'Все работы'}], ['Все работы']
    else:
        all_options = update_works_dropdown(selected_author_value)
        return all_options, selected_value


# Функция обратного вызова для обновления списка "Работы"
@app.callback(
    Output('works-dropdown', 'options'),
    Input('author', 'value'),
)
def update_works_dropdown(selected_value):
    
    if selected_value is None:
        return []

    # Загрузка данных из файла "data_from_json_files.csv"
    data_df = pd.read_csv('./data_from_json_files.csv')

    # Преобразование выбранного ID автора в строку
    selected_value_str = str(selected_value)

    # Разделение значения "Авторы" на список ID авторов
    data_df['Авторы'] = data_df['Авторы'].apply(lambda x: [] if pd.isna(x) else x.split(', '))

    # Фильтрация строк, в которых в списке ID авторов есть выбранный автор
    works = data_df[data_df['Авторы'].apply(lambda x: selected_value_str in x)]

    works_data = works[['Название', 'ID']].drop_duplicates(subset=['ID'])
    # Обрезаем названия работ до первых 15 символов
    works_data['Название'] = works_data['Название'].str[:200]
    
    works_data.columns = ['label', 'value']
    works_data = works_data.to_dict(orient='records')
    print(works_data)
    works_data.insert(0, {'label': "Все работы", 'value': 'Все работы'})
    return works_data



if __name__ == '__main__':
    app.run_server(debug=True)


[{'label': 'Model for Analyzing Impact of Path Loss on eMBB Bit Rate Degradation Under Priority URLLC Transmission in 5G Network', 'value': 71958}]
[{'label': 'Model for Analyzing Impact of Path Loss on eMBB Bit Rate Degradation Under Priority URLLC Transmission in 5G Network', 'value': 71958}]
[{'label': 'Algorithm for Multi-drone Path Planning and Coverage of Agricultural Fields', 'value': 66264}, {'label': 'Разработка сценариев полетных заданий БПЛА в виртуальном полигоне для задач группового управления', 'value': 58990}, {'label': 'Developing a Flight Mission Simulator in the Context of UAVs Group Control', 'value': 59232}, {'label': 'Применение виртуального полигона для формирования полетного задания БПЛА в условиях групповго взаимодействия', 'value': 60582}, {'label': 'Тренажерный комплекс беспилотной авиационной системы (БАС) полевого назначения', 'value': 51578}, {'label': 'Применение виртуальных полигонов для отработки сценариев группового управления бпла мультикоптерного типа